# todo

* do something about the epochTime
* 'FillValue'

In [ ]:
to heigh precision

'_FillValue' and _ChunkSizes

_NCProperties = "version=1|netcdflibversion=4.4.1|hdf5libversion=1.8.17";
Conventions = "CF-1.6";
history = "[2017-08-11 14:24] Created compressed netCDF4 dataset";
command_line = “surfrad_process –f option –c some other options…”


# Questions

- i noticed that data is stored as float32 in the old data ... would prefer 64 ... problem?
- fillValue ... there is this "\_FillValue" parameter which don't know what it exatly mean ... is that actually the fillValue or does that mean that feature is not used ... why the underscore?!?
- not all variables have "standard_name"
- ancillary_variables is applied multiple time -> attribute can only be a string ... made a single string now

# imports 

# parse metadata

In [199]:
def parse_metadata_file(fname = '../data/SURFRAD_QCrad_metadata_kehoe_2018-04-09.txt'):
    def variableblock2attributes(var):
        def clean_variable_attr(variable_line):
            var = variable_line.split('=')[1]
            var = var.strip(' ')
            replacelist = [';', '”', '“', '"']
            for i in replacelist:
                var = var.replace(i, '')#.replace(';').replace('”').replace('“')
            return var
        
        vdic = {}
        vlines = var.split('\n')
        first_line = vlines.pop(0).split()
        if 'global' in first_line:
            vlines.pop(-1)
            vlines.pop(0)
        else:
            vtype = first_line.pop(0)
            vname = first_line.pop(0).replace(';','').replace('(Time=479)', '').replace('(time=479)', '')
            vdic['name'] = vname
            vdic['type'] = vtype
            if len(first_line) != 0:
                vdic['Coordinates'] = first_line.pop(0)

        ancillary_variables = []
        attlist = ['units', 'long_name', 'standard_name', 'comments', 'flag_values', 'flag_meanings', 
                  'title', 'source', 'references', 'institution']
        ignorelist = ['FillValue', 'epoch_time', '_ChunkSizes',
                     '_NCProperties', 'Conventions', 'history', 'command_line',
                     '}']
        for vl in vlines:
            found = ignored = False
            for att in attlist:
                if att in vl:
                    catt = clean_variable_attr(vl)
                    if att in vdic.keys():
                        raise ValueError('This attribute ({}) exists already!!'.format(att))
                    vdic[att] = catt
                    found = True
            for att in ignorelist:
                if att in vl:
                    ignored = True

            if found:
                pass
            elif ignored:
                pass
            elif 'ancillary_variables' in vl:
                ancillary_variables.append(clean_variable_attr(vl))
                vdic['ancillary_variables'] = ancillary_variables
            else:
                raise ValueError('No info found in line: {}'.format(vl))

        # some conditioning
        if 'ancillary_variables' in vdic.keys():
            vdic['ancillary_variables'] = ', '.join(vdic['ancillary_variables'])
        return vdic
    
    # open
    full_dict = {}
    with open(fname, 'r') as rein:
        metadatatxtlines = rein.readlines()

    # conditioning and splitting into blocks
    metadatatxtlines = [line.strip(' ') for line in metadatatxtlines]
    metadatatxt = ''.join(metadatatxtlines)
    metadatablocks = metadatatxt.split('\n\n')

    #get output file name
    full_dict['fname_out'] = metadatablocks.pop(0).split('\n')[0].split()[1].strip('"')
    pop = metadatablocks.pop(0) # no real information in hear
    
    # get global attributes
    mglobalatts = metadatablocks.pop(-1)
#     return mglobalatts
    # the rest are variables
    mvariables = metadatablocks

    variablesdict = {}
    for e,var in enumerate(mvariables):
        try:
            vdic = variableblock2attributes(var)
    #         if 'flag_values' in vdic.keys():
    #             break
        except ValueError as err:
            print(var)
            print('-----------------')
            print(err)
            break
#         try:
        variablesdict[vdic['name']] = {'attributes': vdic}
#         except:
#             print(vdic)
    full_dict['variables'] = variablesdict
    full_dict['global_variables'] = variableblock2attributes(mglobalatts)
    return full_dict

In [200]:
metadata = parse_metadata_file()

In [201]:
metadata

{'fname_out': 'Bondville_IL_1995_May_10.nc',
 'global_variables': {'comments': 'Raw data taken each minute and saved in daily files by test site',
  'institution': 'National Oceanic and Atmospheric Administration (NOAA) - David Skaggs Research Center - Boulder, CO',
  'references': 'Online: https://www.esrl.noaa.gov/gmd/grad/surfrad/index.html',
  'source': 'NOAA/ESRL/GMD/GRAD SURFRAD Data Archive',
  'title': 'QCRad V3 Dataset from NOAA/ESRL/GMD/GRAD SURFRAD Data Archive'},
 'variables': {'QC_flag_01_thresholds_global_shortwave': {'attributes': {'flag_meanings': 'missing_data_or_test_not_possible no_known_issue global_shortwave_irradiance_too_high too_high_user_configurable too_low_extremely_rare too_high_extremely_rare too_low_physical_limits too_high_physical_limits fail_Rayleigh_limit_test tracker_off',
    'flag_values': '-1b 0b, 1b, 2b, 3b, 4b, 5b, 6b, 7b, 8b, 9b',
    'long_name': 'Is global shortwave missing, too high, or too low?',
    'name': 'QC_flag_01_thresholds_global_sho

In [202]:
columntransrules = {'time': None,
 'solar_zenith_angle':                                                  'Z',
 'distance_from_sun':                                                   'AU',
 'best_estimate_shortwave_irradiance':                                  'BESW',
 'total_downwelling_shortwave_irradiance':                              'GSW',
 'diffuse_downwelling_shortwave_irradiance':                            'DIF',
 'direct_downwelling_shortwave_irradiance':                             'DIR',
 'upwelling_shortwave_irradiance':                                      'SWup',
 'downwelling_longwave_irradiance':                                     'LWdn',
 'upwelling_longwave_irradiance':                                       'LWup',
 'atmospheric_temperature':                                             'Ta',
 'relative_humidity':                                                   'RH',
 'surface_air_pressure':                                                'Prs',
 'downwelling_longwave_case_temperature':                               'LWdTc',
 'downwelling_longwave_dome_temperature':                               'LWdTd',
 'upwelling_longwave_case_temperature':                                 'LWuTc',
 'upwelling_longwave_dome_temperature':                                 'LWuTd',
 'QC_flag_01_thresholds_global_shortwave':                              'qc1',
 'QC_flag_02_thresholds_diffuse_shortwave':                             'qc2',
 'QC_flag_03_thresholds_direct_shortwave':                              'qc3',
 'QC_flag_04_thresholds_upwelling_shortwave':                           'qc4',
 'QC_flag_05_thresholds_downwelling_longwave':                          'qc5',
 'QC_flag_06_thresholds_upwelling_longwave':                            'qc6',
 'QC_flag_07_global_shortwave_over_sum':                                'qc7',
 'QC_flag_08_diffuse_over_global_shortwave':                            'qc8',
 'QC_flag_09_upwelling_shortwave_versus_sum':                           'qc9',
 'QC_flag_10_downwelling_longwave_to_temperature':                      'qc10',
 'QC_flag_11_upwelling_longwave_to_temperature':                        'qc11',
 'QC_flag_12_downwelling_longwave_to_upwelling_longwave':               'qc12',
 'QC_flag_13_downwelling_longwave_case_temperature_vs_air_temperature': 'qc13',
 'QC_flag_14_downwelling_longwave_dome_temperature_vs_air_temperature': 'qc14',
 'QC_flag_15_upwelling_longwave_case_temperature_vs_air_temperature':   'qc15',
 'QC_flag_16_upwelling_longwave_dome_temperature_vs_air_temperature':   'qc16',
 'QC_flag_17_downwelling_longwave_case_temperature_vs_dome_temperature':'qc17',
 'QC_flag_18_upwelling_longwave_case_temperature_vs_dome_temperature':  'qc18',
 'QC_flag_19_air':                                                      'qc19',
 'global_shortwave_irloss_flag':                                        'gflg',
 'diffuse_sortwave_irloss_flag':                                        'dflg',
 'downwelling_shortwave_irradiance_assuming_clear_sky':                 'ClrSW',
 'ir_loss_correction_to_diffuse_shortwave':                             'DifCorr',
 'ir_loss_correction_to_global_shortwave':                              'GSWCorr',
 'downwelling_erythemal_uvb_irradiance':                                'UVB_down',
 'photosynthetically_active_radiation':                                 'PAR_dn',
 'wind_speed':                                                          'WindSp',
 'wind_direction':                                                      'WindDr',
 'latitude': None,
 'longitude': None,
 'altitude': None}

In [203]:
for mvar in  metadata['variables']:
    metadata['variables'][mvar]['raw_data_collmn_name'] = columntransrules[mvar]

In [204]:
import json

In [205]:
mdjs = json.dumps(metadata, indent=4)
# print(mdjs)

In [206]:
with open('./metadata.json', 'w') as raus:
    raus.write(mdjs)

# Read data

In [207]:
fname = '../data/bon_19950510.qdat'
data = pd.read_csv(fname,delim_whitespace=True)

# creating timestamp
datetimestr = data.Date.apply(lambda x: '{0:0>8}'.format(x)) + data.Time.apply(lambda x: '{0:0>4}'.format(x))
data.index = pd.to_datetime(datetimestr, format = "%Y%m%d%H%M") 
data.index.name = 'Time'

# removing obsolete columns
data = data.drop(['Date', 'Time'], axis=1)

In [49]:
data

,BESW,GSW,DIF,DIR,SWup,LWdn,LWup,Ta,RH,Prs,...,dflg,Z,AU,ClrSW,DifCorr,GSWCorr,UVB_down,PAR_dn,WindSp,WindDr
Time,,,,,,,,,,,,,,,,,,,,,
1995-05-10 00:00:00,17.9,17.9,18.0,-0.1,1.7,387.4,420.8,295.0,66.1,-9999.9,...,0,80.7,1.00964,120.4,0.0,5.8,1.0,7.9,8.3,173.2
1995-05-10 00:03:00,15.3,15.3,15.5,-0.2,1.8,387.8,420.2,294.9,66.4,-9999.9,...,0,81.3,1.00965,111.9,0.0,5.8,0.9,6.7,7.8,172.7
1995-05-10 00:06:00,12.4,12.4,12.5,-0.2,1.7,388.4,419.7,295.1,66.4,-9999.9,...,0,81.9,1.00965,103.4,0.0,5.4,0.7,5.5,8.0,172.3
1995-05-10 00:09:00,10.7,10.7,10.9,-0.2,1.6,388.6,419.7,294.8,66.6,-9999.9,...,0,82.4,1.00965,95.2,0.0,5.4,0.6,4.6,7.8,172.9
1995-05-10 00:12:00,9.2,9.2,9.4,-0.1,1.2,389.8,419.4,295.1,66.4,-9999.9,...,0,83.0,1.00965,87.0,0.0,4.9,0.5,4.0,8.0,171.3
1995-05-10 00:15:00,8.9,8.9,9.1,-0.1,1.1,389.9,419.3,295.0,66.2,-9999.9,...,0,83.5,1.00965,79.0,0.0,4.9,0.4,3.8,7.9,169.6
1995-05-10 00:18:00,8.1,8.1,8.3,-0.1,1.0,390.2,419.3,294.9,66.2,-9999.9,...,0,84.0,1.00965,71.1,0.0,4.5,0.4,3.6,7.9,166.7
1995-05-10 00:21:00,8.7,8.7,8.9,-0.1,1.4,390.6,418.7,294.8,66.6,-9999.9,...,0,84.6,1.00965,63.4,0.0,4.5,0.4,3.7,7.7,167.3
1995-05-10 00:24:00,6.8,6.8,6.9,-0.1,1.2,392.2,418.6,294.7,67.0,-9999.9,...,0,85.1,1.00965,55.8,0.0,4.1,0.4,3.1,8.1,167.8


# Create dataset

In [208]:
ds = xr.Dataset()

for var in metadata['variables']:
    if var == 'time':
        vartime = metadata['variables'][var]
        continue
    vardic = metadata['variables'][var]
    dat_col_name = vardic['raw_data_collmn_name']
    if dat_col_name:
        ds[var] = data.loc[:,dat_col_name]
    else:
        if var == 'latitude':
            ds[var] = 40.05192
        elif var == 'longitude':
            ds[var] = 271.62692
        elif var == 'altitude':
            ds[var] = 230
    for atr in vardic['attributes']:
        if atr == 'name':
            continue
        ds[var].attrs[atr] = vardic['attributes'][atr]   

for gv in metadata['global_variables']:
    ds.attrs[gv] = metadata['global_variables'][gv]

for atr in vartime['attributes']:
    if atr in ['name', 'units', 'type']:
            continue
    ds.Time.attrs[atr] = vartime['attributes'][atr]

In [209]:
ds.to_netcdf(metadata['fname_out'])

In [210]:
ds

<xarray.Dataset>
Dimensions:                                                               (Time: 480)
Coordinates:
  * Time                                                                  (Time) datetime64[ns] 1995-05-10 ...
Data variables:
    solar_zenith_angle                                                    (Time) float64 80.7 ...
    distance_from_sun                                                     (Time) float64 1.01 ...
    best_estimate_shortwave_irradiance                                    (Time) float64 17.9 ...
    total_downwelling_shortwave_irradiance                                (Time) float64 17.9 ...
    diffuse_downwelling_shortwave_irradiance                              (Time) float64 18.0 ...
    direct_downwelling_shortwave_irradiance                               (Time) float64 -0.1 ...
    upwelling_shortwave_irradiance                                        (Time) float64 1.7 ...
    downwelling_longwave_irradiance                                     

# structure of nc-files in old script collection

In [152]:
fname = '../scripts/Radiation NetCDF Scripts/NetCDF_output/Bondville_IL_1995_01_01.nc'
dsold = xr.open_dataset(fname, decode_times=False)

In [ ]:
40.05192, 271.62692, 230

In [153]:
dsold

<xarray.Dataset>
Dimensions:                                                   (Time: 479)
Coordinates:
  * Time                                                      (Time) int32 3 ...
Data variables:
    year                                                      (Time) int32 ...
    month                                                     (Time) int32 ...
    day                                                       (Time) int32 ...
    latitude                                                  (Time) float32 ...
    longitude                                                 (Time) float32 ...
    altitude                                                  (Time) int32 ...
    best_estimate_shortwave                                   (Time) float32 ...
    total_shortwave                                           (Time) float32 ...
    diffuse_downwelling_shortwave_flux_in_air                 (Time) float32 ...
    direct_downwelling_shortwave_flux_in_air                  (Time) float32 .

In [91]:
for k in ds.variables.keys():
    print(k)

Time
year
month
day
latitude
longitude
altitude
best_estimate_shortwave
total_shortwave
diffuse_downwelling_shortwave_flux_in_air
direct_downwelling_shortwave_flux_in_air
upwelling_shortwave_flux_in_air
downwelling_longwave_flux_in_air
upwelling_longwave_flux_in_air
air_temperature
relative_humidity
surface_air_pressure
downwelling_longwave_case_temperature
downwelling_longwave_dome_temperature
upwelling_longwave_case_temperature
upwelling_longwave_dome_temperature
qc01
qc02
qc03
qc04
qc05
qc06
qc07
qc08
qc09
qc10
qc11
qc12
qc13
qc14
qc15
qc16
qc17
qc18
qc19
g_flag
d_flag
solar_zenith_angle
distance_from_sun
downwelling_shortwave_flux_in_air_assuming_clear_sky
ir_loss_correction_to_diffuse_shortwave
ir_loss_correction_to_global_shortwave
downwelling_erythemal_uvb_flux_in_air
surface_downwelling_photosynthetic_radiative_flux_in_air
wind_speed
wind_from_direction
